In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

# Parameters

In [3]:
# import pyepo
# generate data
grid = (3,3) # grid size
num_train = 50 # number of training data
num_feat = 5 # size of feature
num_test = 1000
deg = 1.0 # polynomial degree
e = 10 # scale of normal std or the range of uniform. For the error term
lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Test_0324' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("grandparent_directory:", grandparent_directory)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_coef_seed="+str(coef_seed)+"/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

grandparent_directory: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Code_MacBook
DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Test_0324DDR_Data_Generation/


# Generate Data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,seed_all,num_test, num_train, alpha,mis,data_generation_process):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # print("W_star = ",W_star[0,:])
    W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,version = 0) 

    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for seed in seed_all:
        DataPath_seed = DataPath +"Seed="+str(seed)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        # #  ****** Data generation *********
        x_test_all[seed], c_test_all[seed], x_train_all[seed], c_train_all[seed], W_star_all[seed] = data_gen.generate_samples(seed,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, thres = 10, 
                                version = data_generation_process, x_dist = 'normal', e_dist = 'normal', x_low = 0, x_up = 2, x_mean = 2, x_var = 0.25, bump = 100) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

# Robustness Formulation

### Define network

In [6]:
def _getArcs(grid):
    arcs = []
    for i in range(grid[0]):
        # edges on rows
        for j in range(grid[1] - 1):
            v = i * grid[1] + j
            arcs.append((v, v + 1))
        # edges in columns
        if i == grid[0] - 1:
            continue
        for j in range(grid[1]):
            v = i * grid[1] + j
            arcs.append((v, v + grid[1]))

    arc_index_mapping = {}
    for i in range(len(arcs)):
        arc = arcs[i]
        arc_index_mapping[arc] = i

    return arcs,arc_index_mapping

In [7]:
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt

## Solve the model with OLS estimation, the objective can be used as target

$\min \frac{1}{N} \sum_{n \in [N]}[\mu y_{n}^{\top}\tilde{z}_{n} + (1 - \mu )y_{n}^\top f(\tilde{x}_{n},w^{OLS})]$

In [8]:
def Solve_DDR_inner(arcs,arc_index_mapping,mu,num_train,num_arcs,c_train,x_train,W_ols,w0_ols,grid):
    model = ro.Model()            # create an RSOME model
    y = model.dvar((num_train,num_arcs))
    t = model.dvar()
    weight_sum = model.dvar(num_train)                   

    for i in range(num_train):
        model.st(weight_sum[i] == mu * (y[i,:] @ c_train[i,:]) + (1 - mu) *  y[i,:] @ (W_ols @ x_train[i,:] + w0_ols))
    model.st(t == weight_sum.sum() * (1/num_train))
    model.st(y >= 0)

    # shortest path constraints
    for n in range(num_train):
        for i in range(grid[0]):
            for j in range(grid[1]):
                v = i * grid[1] + j
                expr = 0
                for arc in arcs:
                    # flow in
                    if v == arc[1]:
                        expr += y[n,arc_index_mapping[arc]]
                    # flow out
                    elif v == arc[0]:
                        expr -= y[n,arc_index_mapping[arc]]
                # source
                if i == 0 and j == 0:
                    model.st(expr == -1)
                # sink
                elif i == grid[0] - 1 and j == grid[0] - 1:
                    model.st(expr == 1)
                # transition
                else:
                    model.st(expr == 0)

    model.min(t) 

    model.solve(grb)
    y_sol = y.get()
    obj = model.obj()
    # print("obj = ",obj)

    # i = 1
    # path = []
    # for j in range(num_arcs):
    #     if y_sol[i,j] > 0:
    #         path.append(arcs[j])
    # print("The",i,"Sample: shortest path = ",path)
    return obj,y_sol

## Solve Sub-RDDR

In [9]:
def solve_sub_rddr(arcs,arc_index_mapping,mu,rho,num_train,num_arcs,c_train,x_train,grid,is_binary):
    model = ro.Model()            # create an RSOME model
    if is_binary:
        y = model.dvar((num_train,num_arcs),vtype='B')
    else:
        y = model.dvar((num_train,num_arcs))
    t = model.dvar()
    gamma = model.dvar()
    delta = model.dvar((num_train,num_arcs))
    delta_vector = model.dvar(num_train * num_arcs)
    weight_sum = model.dvar(num_train)                   

    for i in range(num_train):
        model.st(weight_sum[i] == mu * (y[i,:] @ c_train[i,:]) + (1 - mu) *  delta[i,:] @ c_train[i,:])
    model.st(y >= 0)
    model.st(t >= weight_sum.sum() * (1/num_train) + ((1-mu)/num_train)*rho*gamma)

    for j in range(num_arcs):
        model.st(delta[:,j].sum() == y[:,j].sum())
        model.st(x_train.T @ delta[:,j] == x_train.T @ y[:,j])

    index = 0
    for i in range(num_train):
        for j in range(num_arcs):
            model.st(delta_vector[index] == delta[i,j])
            index = index + 1
    model.st(rso.norm(delta_vector,2) <= gamma)        # a constraint with 2-norm terms
    model.st(gamma >= 0)


    # shortest path constraints
    for n in range(num_train):
        for i in range(grid[0]):
            for j in range(grid[1]):
                v = i * grid[1] + j
                expr = 0
                for arc in arcs:
                    # flow in
                    if v == arc[1]:
                        expr += y[n,arc_index_mapping[arc]]
                    # flow out
                    elif v == arc[0]:
                        expr -= y[n,arc_index_mapping[arc]]
                # source
                if i == 0 and j == 0:
                    model.st(expr == -1)
                # sink
                elif i == grid[0] - 1 and j == grid[0] - 1:
                    model.st(expr == 1)
                # transition
                else:
                    model.st(expr == 0)

    model.min(t) 
    model.solve(cpt,display=False,log=False)
    y_sol = y.get()
    obj = model.obj()
    # print("obj = ",obj)
    return obj,y_sol

## RobustW

In [10]:
def solve_robustW(mu,rho,num_train,num_arcs,num_feat,c_train,x_train,y_sol):
    model = ro.Model()            # create an RSOME model

    W_DDR = model.dvar((num_arcs,num_feat))
    W0_DDR = model.dvar(num_arcs)
    norm_vector = model.dvar(num_train*num_arcs)
    t = model.dvar()
    weight_sum = model.dvar(num_train)                   

    for i in range(num_train):
        model.st(weight_sum[i] == mu * (y_sol[i,:] @ c_train[i,:]) + (1 - mu) *  y_sol[i,:] @ (W_DDR @ x_train[i,:] + W0_DDR))
    model.st(t == weight_sum.sum() * (1/num_train))

    for i in range(num_train):
        model.st(norm_vector[i*num_arcs:(i+1)*num_arcs] == c_train[i,:] - W_DDR @ x_train[i,:] - W0_DDR)
    model.st(rso.norm(norm_vector,2) <= rho)

    model.max(t) 

    model.solve(cpt,display=False,log=False)
    obj = model.obj()
    # print("obj = ",obj)
    W_DDR_val = W_DDR.get()
    W0_DDR_val = W0_DDR.get()
    return obj,W_DDR_val,W0_DDR_val

# Compute out-of-sample performance

In [11]:
arcs,arc_index_mapping = _getArcs(grid)
num_arcs = len(arcs)

seed_all = np.arange(1,10)
# obtain data
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,seed_all,num_test, num_train, alpha,mis,data_generation_process)

In [23]:
from OLS import ols_method
ols_method_obj = ols_method()
from Peformance import performance_evaluation
perfs = performance_evaluation()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}; rho_lb_all = {}
cost_OLS_all = {}; cost_Oracle_all = {}

for seed in seed_all:
    # compute OLS performance
    W_ols_all[seed], w0_ols_all[seed], t_ols_all[seed], obj_ols_all[seed] = ols_method_obj.ols_solver("",x_train_all[seed], c_train_all[seed])
    cost_OLS_all[seed] = perfs.compute_Cost_with_Prediction(arcs,w0_ols_all[seed],W_ols_all[seed], grid,c_test_all[seed],x_test_all[seed])
    cost_Oracle_all[seed] = perfs.compute_Oracel_Cost(arcs, grid,c_test_all[seed])

    # obtain the lowest rho
    # is_binary = False
    rho_lb_all[seed] =  np.sqrt(obj_ols_all[seed] * num_train) 

In [24]:

def obtain_out_of_sample_performance(seed,mu,rho_lb,rho_coef_all,is_binary,arcs,arc_index_mapping,grid,x_train_all,c_train_all,x_test_all,c_test_all,cost_OLS,cost_Oracle,perfs):
    x_train = x_train_all[seed]
    c_train = c_train_all[seed]
    x_test = x_test_all[seed]
    c_test = c_test_all[seed]

    regret_arr = np.zeros(len(rho_coef_all))
    rho_index = 0
    for rho_coef in rho_coef_all:
        rho = rho_lb + rho_coef * rho_lb
    
        obj_rddr,y_sol_rddr = solve_sub_rddr(arcs,arc_index_mapping,mu,rho,num_train,num_arcs,c_train,x_train,grid,is_binary)
        obj,W_DDR,W0_DDR = solve_robustW(mu,rho,num_train,num_arcs,num_feat,c_train,x_train,y_sol_rddr)
        cost_DDR = perfs.compute_Cost_with_Prediction(arcs,W0_DDR,W_DDR, grid,c_test,x_test)
        regret = (np.nanmean(cost_OLS) - np.nanmean(cost_DDR) )/(np.nanmean(cost_OLS) - np.nanmean(cost_Oracle)) * 100
        # print("seed=",seed,",mu=",mu,"rho=",rho,",regret=",np.round(regret,4))
        regret_arr[rho_index] = regret
        rho_index = rho_index +1
    return regret_arr

In [26]:
mu = 0.9
rho_coef_all = np.arange(0.0,0.01,0.001)
is_binary = False
regret_all = {}
for seed in seed_all:
    regret_all[seed,mu] = obtain_out_of_sample_performance(seed,mu,rho_lb_all[seed],rho_coef_all,is_binary,arcs,arc_index_mapping,grid,x_train_all,c_train_all,x_test_all,c_test_all,cost_OLS_all[seed],cost_Oracle_all[seed],perfs)
    print("seed=",seed,",mu=","regret reduction=",mu,np.round(regret_all[seed,mu],4))

seed= 1 ,mu= 0.9 [-0.0019 -0.1743 -0.2913 -0.3863 -0.4903 -0.5778 -0.5942 -0.5977 -0.7287
 -0.7206]
seed= 2 ,mu= 0.9 [ 0.0157 -0.0199  0.1612 -0.0014  0.0427  0.1889  0.3125 -0.1027 -0.312
 -0.3616]
seed= 3 ,mu= 0.9 [ 0.     -0.1973 -0.3986 -0.3983 -0.5831 -0.6918 -0.65   -0.6695 -0.6948
 -0.6128]
seed= 4 ,mu= 0.9 [0.     0.0626 0.1355 0.2046 0.4023 0.6728 0.7917 0.8667 1.0274 1.1376]
seed= 5 ,mu= 0.9 [ 0.0081  0.4371  0.5617  0.6825  0.4423  0.4144  0.282  -0.0781 -0.0592
 -0.1689]
seed= 6 ,mu= 0.9 [-0.0014  0.0159 -0.1927 -0.3533 -0.4456 -0.2342 -0.0657 -0.0776  0.1227
  0.0705]
seed= 7 ,mu= 0.9 [ 0.0174 -0.1483 -0.3735 -0.2396 -0.2997 -0.3003 -0.2603 -0.2294 -0.4788
 -0.639 ]
seed= 8 ,mu= 0.9 [-0.0219  0.8786  0.9337  1.389   1.4608  1.967   2.161   2.1011  1.9713
  1.8977]
seed= 9 ,mu= 0.9 [-0.0027 -0.1271 -0.1705 -0.103  -0.1774 -0.2314 -0.3167 -0.4135 -0.3666
 -0.4807]


In [27]:
regret_avg = np.zeros(len(rho_coef_all))
for seed in seed_all:
    regret_avg = regret_avg + regret_all[seed,mu]
regret_avg = regret_avg/(len(seed_all))
print("regret_avg=",np.round(regret_avg,4))

regret_avg= [0.0015 0.0808 0.0406 0.0883 0.0391 0.1342 0.1845 0.0888 0.0535 0.0136]
